# OpenPredict API examples

Example calls to the OpenPredict Smart API accessible at https://openpredict.semanticscience.org

## Get predictions for a list of MONDO diseases

Example to convert multiple MONDO IDs to OMIM with Translator NodeNormalization API, then query the OpenPredict API, for predicted drugs.

In [1]:
import json 
import requests

max_predictions_returned = 5
mondo_ids_list = ["MONDO:0018874", "MONDO:0008734", "MONDO:0004056", "MONDO:0005499", "MONDO:0006256", "MONDO:0006143", "MONDO:0019087", "MONDO:0002271", "MONDO:0003093", "MONDO:0018177", "MONDO:0010150", "MONDO:0017885", "MONDO:0005005", "MONDO:0017884", "MONDO:0007256", "MONDO:0005061", "MONDO:0005097", "MONDO:0018905", "MONDO:0005065", "MONDO:0006046", "MONDO:0006047", "MONDO:0004974", "MONDO:0005082", "MONDO:0002169", "MONDO:0005089", "MONDO:0005012", "MONDO:0005036", "MONDO:0010108", "MONDO:0006456", "MONDO:0015075", "MONDO:0006485", "MONDO:0000553", "MONDO:0006486", "MONDO:0004967", "MONDO:0005170", "MONDO:0005072", "MONDO:0008433", "MONDO:0004163", "MONDO:0000554", "MONDO:0005580", "MONDO:0004093", "MONDO:0000448"]

# First query Translator NodeNormalization API to convert MONDO IDs to OMIM IDs
resolve_curies = requests.get('https://nodenormalization-sri.renci.org/get_normalized_nodes',
                     params={'curie': mondo_ids_list})

# Get corresponding OMIM IDs for MONDO IDs if match
resp = resolve_curies.json()
for mondo_id, alt_ids in resp.items():
    for alt_id in alt_ids['equivalent_identifiers']:
        if str(alt_id['identifier']).startswith('OMIM'):
            print('🗺 Mapped ' + mondo_id + ' - "' + alt_ids['id']['label'] + '" to ' + alt_id['identifier'])
            print('🔎 Searching drug predictions for ' + alt_id['identifier'])
            # Query OpenPredict API with OMIM IDs using /predict call (faster than TRAPI /query)
            get_predictions = requests.get('https://openpredict.semanticscience.org/predict',
                     params={
                         'disease_id': alt_id['identifier'],
                         'n_results': max_predictions_returned
                     }).json()
            # Display predictions found
            if 'hits' in get_predictions:
                print('🔮 Drug predictions found:')
                for prediction in get_predictions['hits']:
                    # print(prediction)
                    predicted_label = prediction['id']
                    if 'label' in prediction.keys():
                        predicted_label = predicted_label + ' - ' + prediction['label']
                    print(predicted_label + ' - score: ' + str(prediction['score']))
            else:
                print('❌️ No drug prediction found')
            print('')

🗺 Mapped MONDO:0018874 - "acute myeloid leukemia" to OMIM:601626
🔎 Searching drug predictions for OMIM:601626
🔮 Drug predictions found:
DRUGBANK:DB00445 - 4'-epidoxorubicin - score: 0.9961497737170921
DRUGBANK:DB00523 - 9-cis-retinoic acid - score: 0.9874284396192737
DRUGBANK:DB00755 - all-trans-retinoic acid - score: 0.9743126272421361
DRUGBANK:DB01177 - idarubicin - score: 0.8720287460212743
DRUGBANK:DB00694 - daunorubicin - score: 0.8690425579912955

🗺 Mapped MONDO:0008734 - "adrenocortical carcinoma, hereditary" to OMIM:202300
🔎 Searching drug predictions for OMIM:202300
❌️ No drug prediction found

🗺 Mapped MONDO:0010150 - "head and neck squamous cell carcinoma" to OMIM:275355
🔎 Searching drug predictions for OMIM:275355
❌️ No drug prediction found

🗺 Mapped MONDO:0017884 - "papillary renal cell carcinoma" to OMIM:605074
🔎 Searching drug predictions for OMIM:605074
❌️ No drug prediction found

🗺 Mapped MONDO:0007256 - "hepatocellular carcinoma" to OMIM:114550
🔎 Searching drug pred

## TRAPI `/query`

Send a query described using the Translator Reasoner API standard to OpenPredict:

In [2]:
data = json.dumps({
  "message": {
    "n_results": "2",
    "query_graph": {
      "edges": {
        "e01": {
          "object": "n1",
          "predicate": "biolink:treated_by",
          "subject": "n0"
        }
      },
      "nodes": {
        "n0": {
          "category": "biolink:Drug",
          "id": "DRUGBANK:DB00394"
        },
        "n1": {
          "category": "biolink:Disease"
        }
      }
    },
    "query_options": {
      "min_score": "0.5"
    }
  }
})
headers = {'Content-type': 'application/json'}
predict_drugs = requests.post('https://openpredict.semanticscience.org/query',
                     data=data, headers=headers)
print(json.dumps( predict_drugs.json(), indent = 2))

{
  "knowledge_graph": {
    "edges": {
      "e0": {
        "association_type": "biolink:ChemicalToDiseaseOrPhenotypicFeatureAssociation",
        "has_confidence_level": 0.8267106697312154,
        "object": "DRUGBANK:DB00394",
        "predicate": "biolink:treated_by",
        "provided_by": "Translator OpenPredict model openpredict_baseline",
        "subject": "OMIM:246300"
      },
      "e1": {
        "association_type": "biolink:ChemicalToDiseaseOrPhenotypicFeatureAssociation",
        "has_confidence_level": 0.8246895250693298,
        "object": "DRUGBANK:DB00394",
        "predicate": "biolink:treated_by",
        "provided_by": "Translator OpenPredict model openpredict_baseline",
        "subject": "OMIM:181460"
      },
      "e10": {
        "association_type": "biolink:ChemicalToDiseaseOrPhenotypicFeatureAssociation",
        "has_confidence_level": 0.6958029445317564,
        "object": "DRUGBANK:DB00394",
        "predicate": "biolink:treated_by",
        "provided_by"

## Simpler `/predict` operation

This operation allow to quickly get predicted drug/disease for the given drug/disease (same as the TRAPI operation, but simpler to query)

Example providing a `disease_id`, we can also provide a `drug_id` in the params: `{'drug_id':"DRUGBANK:DB00394"}` 

In [3]:
# For a disease
predict_diseases = requests.get('https://openpredict.semanticscience.org/predict',
                     params={'disease_id':"OMIM:268220", 
                             'model_id':'openpredict_baseline',
                             'n_results': 2})
print(json.dumps( predict_diseases.json(), indent = 2))

{
  "count": 2,
  "hits": [
    {
      "id": "DRUGBANK:DB00445",
      "label": "4'-epidoxorubicin",
      "score": 0.9961497737170921,
      "type": "drug"
    },
    {
      "id": "DRUGBANK:DB00218",
      "label": "moxifloxacin",
      "score": 0.8557240065188978,
      "type": "drug"
    }
  ]
}


## Translator identifiers resolution

Example query to convert OMIM ID to MONDO ID using the Translator NodeNormalization API:

Translator ID preferences:
* [For Disease](https://github.com/biolink/biolink-model/blob/master/biolink-model.yaml#L2853)
* [Chemical substance](https://github.com/biolink/biolink-model/blob/master/biolink-model.yaml#L3097) 
* [Drug](https://github.com/biolink/biolink-model/blob/master/biolink-model.yaml#L3119)

Resolve CURIEs using Translator API services: 

In [4]:
resolve_curies = requests.get('https://nodenormalization-sri.renci.org/get_normalized_nodes',
                     params={'curie':["OMIM:268220", "DRUGBANK:DB00394", "HP:0007354", "HGNC:613"]})
print( json.dumps( resolve_curies.json(), indent = 2))

{
  "OMIM:268220": {
    "id": {
      "identifier": "MONDO:0009994",
      "label": "alveolar rhabdomyosarcoma (disease)"
    },
    "equivalent_identifiers": [
      {
        "identifier": "MONDO:0009994",
        "label": "alveolar rhabdomyosarcoma (disease)"
      },
      {
        "identifier": "DOID:4051"
      },
      {
        "identifier": "OMIM:268220"
      },
      {
        "identifier": "ORPHANET:99756"
      },
      {
        "identifier": "UMLS:C0206655"
      },
      {
        "identifier": "MESH:D018232"
      },
      {
        "identifier": "MEDDRA:10065867"
      },
      {
        "identifier": "NCIT:C3749"
      },
      {
        "identifier": "SNOMEDCT:404053004"
      },
      {
        "identifier": "SNOMEDCT:63449009"
      },
      {
        "identifier": "HP:0006779",
        "label": "Alveolar rhabdomyosarcoma"
      }
    ],
    "type": [
      "biolink:Disease",
      "biolink:DiseaseOrPhenotypicFeature",
      "biolink:BiologicalEntity",
      "bi